<a href="https://colab.research.google.com/github/wiltacca/Portfolio/blob/main/ITI104_Dimensionality_Reduction_Lab_v1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ITI104 Machine Learning Algorithms
# Dimensionality Reduction

## Introduction <br>
In this practical, we will experiment with three main applications of dimensionality reduction techniques, i.e. <br>
*  As a preprocessing step to improve the performance of model training <br>
*  Data compression <br>
*  Data visualization <br> <br>
The dimensionality reduction techniques covered are: Singular Value Decomposition (Scikit-Learn's `PCA` class, NumPy's `svd()` function), Eigen analysis (NumPy's `eig()` function), and Manifold Learning (Scikit-Learn's `KernelPCA` class). <br> <br>
We will use the Iris, MNIST, and Swiss roll datasets for the experiments.

In [ ]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

## 1. PCA and `svd()` with Iris dataset <br>
-  In this section, we will learn the basics of Scikit-Learn's `PCA` class and NumPy's `svd()` function <br>
-  We will visualize the Iris dataset by reducing its dimensions from 4 to 2 and plotting it on a scatter plot <br>
-  We will try to make some interpretation on the resulting visualization

Load and explore the Iris dataset

In [ ]:
import seaborn as sns
iris = sns.load_dataset('iris')

In [ ]:
iris.shape

In [ ]:
iris.head()

In [ ]:
iris.tail()

Take a look at how the data instances are distributed in the feature space

In [ ]:
sns.pairplot(iris, hue='species')

Observe that Setosa is the most separable

Separate the dataset into features and the label

In [ ]:
X = iris.drop('species', axis=1)
y = iris['species']

In [ ]:
X.shape

In [ ]:
y.shape

Apply PCA to the dataset (feature columns only) to reduce its dimensions from 4 to 2

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=2)
X_reduced = pca.fit_transform(X)

In [ ]:
pca.n_components_

What ratio of the variance in the original data is accounted for by the first principal component? What ratio by the second?

In [ ]:
pca.explained_variance_ratio_

In total, what is the percentage of the variance in the original data that is preserved by PCA?

In [ ]:
np.sum(pca.explained_variance_ratio_)

What are the coordinates of the two principal components?

In [ ]:
pca.components_

First principal component

In [ ]:
pca.components_[0]

Another way of reading the first principal component

In [ ]:
pca.components_.T[ : , 0]

Principal components are unit vectors

In [ ]:
np.linalg.norm(pca.components_[0])

In [ ]:
np.linalg.norm(pca.components_[1])

Look's look at the dataset in the 2-dimensional subspace

In [ ]:
X_reduced.shape

In [ ]:
X_reduced

Pair plot of the reduced dataset

In [ ]:
X_reduced_df = pd.DataFrame(X_reduced, columns=["1st principal component", "2nd principal component"])

In [ ]:
X_reduced_df.head()

In [ ]:
iris_reduced = pd.concat([X_reduced_df, y], axis=1)

In [ ]:
iris_reduced.head()

In [ ]:
sns.pairplot(iris_reduced, hue='species')

Scatter plot of the reduced dataset

In [ ]:
sns.scatterplot(iris_reduced['1st principal component'], iris_reduced['2nd principal component'], hue=iris_reduced['species'])

How does the Iris dataset distribution look like in the 2-dimensional space? <br>
Are the three species well separated?

Instead of Scikit-Learn's `PCA` class, try the dimensionality reduction process using Numpy's `svd()` function

## 2. PCA with MNIST dataset
The MNIST dataset is a set of 70,000 small images (28x28) of handwritten digits, divided into 60,000 for training and 10,000 for testing. Each image is labeled with the digit that it represents. <br>
-  In this section, we will first explore the MNIST dataset <br>
-  We will compress the MNIST dataset by dimensionality reduction with different ratios of preserved variance. We will then try to recover the original dataset by decompressing it and examine the errors produced in the process <br>
-  Dimensionality reduction is often used as a preprocessing step before training a machine learning model. We will examine the effect of dimensionality reduction on the performance of a classifier trained on the original dataset and the dataset with dimensions reduced <br>
<br>
(Some parts of the code in this section are adopted from Reference [1])

Load the MNIST dataset

In [ ]:
mnist_train = pd.read_csv("mnist_train.csv")
mnist_test = pd.read_csv("mnist_test.csv")

In [ ]:
mnist_train.shape, mnist_test.shape

In [ ]:
mnist_train.head()

Separate the label from features as we usually do

In [ ]:
X_train = mnist_train.drop("label", axis=1)
y_train = mnist_train["label"]
X_test = mnist_test.drop("label", axis=1)
y_test = mnist_test["label"]

In [ ]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

Let's explore the MNIST dataset using the test set

In [ ]:
digits = X_test.values

In [ ]:
digits

Pick a digit

In [ ]:
some_digit = digits[5600]

In [ ]:
some_digit

In [ ]:
def plot_digit(data):
    image = data.reshape(28, 28)
    plt.imshow(image, cmap = mpl.cm.binary,
               interpolation="nearest")
    plt.axis("off")

In [ ]:
plot_digit(some_digit)

It's a 5.  Let's verify

In [ ]:
mnist_test.iloc[5600, 0]

View the images of some sample digits from 0 to 9

In [ ]:
def plot_digits(instances, images_per_row=10, **options):
    size = 28
    images_per_row = min(len(instances), images_per_row)
    images = [instance.reshape(size,size) for instance in instances]
    n_rows = (len(instances) - 1) // images_per_row + 1
    row_images = []
    n_empty = n_rows * images_per_row - len(instances)
    images.append(np.zeros((size, size * n_empty)))
    for row in range(n_rows):
        rimages = images[row * images_per_row : (row + 1) * images_per_row]
        row_images.append(np.concatenate(rimages, axis=1))
    image = np.concatenate(row_images, axis=0)
    plt.imshow(image, cmap = mpl.cm.binary, **options)
    plt.axis("off")

In [ ]:
plt.figure(figsize=(9,9))
example_images = np.r_[digits[:1000:100], digits[1000:2000:100], digits[2200:3200:100], digits[3200:4200:100], digits[4200:5200:100], digits[5150:6030:95], digits[6150:7000:93], digits[7000:7850:98], digits[8000:8990:98], digits[9000::100]]
plot_digits(example_images, images_per_row=10)
plt.show()

MNIST compression <br>
One of the applications of dimensionality reduction is data compression

Combine `X_train` and `X_test` into one dataset (without label) for dimensionality reduction

In [ ]:
X = pd.concat([X_train, X_test], axis=0)

In [ ]:
X.shape

Apply PCA to reduce the dimensionality of the combined training and test sets <br>
We want to retain at least 95% of the variance

We can set `n_components` to be a float between 0.0 and 1.0 to indicate the ratio of variance we wish to preserve

In [ ]:
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X)

In [ ]:
pca.n_components_

In [ ]:
np.sum(pca.explained_variance_ratio_)

If we know the number of dimensions to reduce to, we can just specify it directly as in the code below

In [ ]:
pca = PCA(n_components = 154)
X_reduced = pca.fit_transform(X)

In [ ]:
X_reduced.shape

With the dimensions reduced from 784 to 154, the dataset is now less than 20% of its original size while still preserving most of the variance

Inverse transformation (performed by the `inverse_transform()` method) reverses the process of dimensionality reduction and "decompresses" the reduced dataset back to the original dimensions 

In [ ]:
X_recovered = pca.inverse_transform(X_reduced)

In [ ]:
X_reduced.shape

In [ ]:
X_reduced

In [ ]:
X_recovered.shape

In [ ]:
X_recovered

Did we get back exactly the original X?

Now, plot the compressed images of the digits that we have just plotted a few steps ago

In [ ]:
plt.figure(figsize=(9,9))
example_images = np.r_[X_recovered[60000:61000:100], X_recovered[61000:62000:100], X_recovered[62200:63200:100], X_recovered[63200:64200:100], X_recovered[64200:65200:100], X_recovered[65150:66030:95], X_recovered[66150:67000:93], X_recovered[67000:67850:98], X_recovered[68000:68990:98], X_recovered[69000::100]]
plot_digits(example_images, images_per_row=10)
plt.show()

Any noticeable degradation in image quality?

Compute the reconstruction error (i.e. the mean squared distance between the original data and the reconstructed data)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(X_recovered, X.values)

What if we compress the original dataset such that only 60% of the variance is preserved. What will be the resulting quality of the compressed images?

Train a SVM classifier (`LinearSVC()` or `SVC()`) on the original MNIST dataset <br>
Time how long the training takes (you may use the magic function `%timeit`, e.g. `%timeit my_clf.fit(X,y)`), <br>
and evaluate the model on the original test set <br>
In order for the training not to take too long, reduce the training set size to 3000 and test set size to 1000 <br>
`X_train_samp = X_train[ : : 20]` <br>
`y_train_samp = y_train[ : : 20]` <br>
`X_test_samp = X_test[ : : 10]` <br>
`y_test_samp = y_test[ : : 10]`

Now, train the same classifier on the dataset with reduced dimensions (take 3000 and 1000 corresponding samples for training and test respectively) <br>
`X_train_reduced = pca.fit_transform(X_train)` <br>
`X_train_reduced_samp = X_train_reduced[ : : 20]` <br>
`X_test_reduced = pca.transform(X_test)` <br>
`X_test_reduced_samp = X_test_reduced[ : : 10]` <br>
Take note of the training time. How does it compare to the previous classifier? <br>
What about the performance of the new model?

## 3. Eigen analysis
-  In this section, we will perform an eigen analysis using NumPy's `eig()` function
-  The steps follow closely those presented in the lecture notes <br>
(Some parts of the code in this section are adopted from References [1] and [2])

Build a 3D dataset for this experiment (the 3D dataset is the one shown in slide 8 of the lecture notes)

In [ ]:
np.random.seed(4)
m = 60
w1, w2 = 0.1, 0.3
noise = 0.1

angles = np.random.rand(m) * 3 * np.pi / 2 - 0.5
X = np.empty((m, 3))
X[:, 0] = np.cos(angles) + np.sin(angles)/2 + noise * np.random.randn(m) / 2
X[:, 1] = np.sin(angles) * 0.7 + noise * np.random.randn(m) / 2
X[:, 2] = X[:, 0] * w1 + X[:, 1] * w2 + noise * np.random.randn(m)

In [ ]:
X.shape

In [ ]:
X

PCA using NumPy's `eig()` function

Eigen function requires the dataset to be mean centered

In [ ]:
X_centered = X - X.mean(axis=0)

In [ ]:
X_centered.shape

Compute the covariance matrix of the dataset

In [ ]:
cov_mat = (X_centered).T.dot(X_centered) / (X.shape[0]-1)

In [ ]:
cov_mat.shape

Compute the eigenvalues and eigenvectors of the covariance matrix using `eig()` function

In [ ]:
eig_vals, eig_vecs = np.linalg.eig(cov_mat)

In [ ]:
eig_vals.shape

In [ ]:
eig_vals

In [ ]:
eig_vals.sum()

In [ ]:
eig_vecs.shape

In [ ]:
eig_vecs  # Columns are eigen vectors

Eigen vectors are unit vectors

In [ ]:
print([np.linalg.norm(eig_vecs[:,i]) for i in range(len(eig_vals))])

Use the eigenvalues to select the most important dimensions in the new data space

In [ ]:
# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
# Note that eig_vals returned by eig() function are not necessarily ordered
eig_pairs.sort(key=lambda x: x[0], reverse=True)

In [ ]:
eig_pairs

Assuming that we want to retain at least 97% of the total variance, how many dimensions do we need to keep?

In [ ]:
# Only keep a certain number of eigenvectors
# based on the "explained variance ratio"
# which tells us how much information (variance) is explained by each eigenvector

exp_var_percentage = 97  # Threshold of 97% explained variance

tot = sum(eig_vals)
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

num_vec_to_keep = 0

for index, percentage in enumerate(cum_var_exp):
  # print(index, percentage, exp_var_percentage, num_vec_to_keep)
  if percentage > exp_var_percentage:
    num_vec_to_keep = index + 1
    break

In [ ]:
num_vec_to_keep

In [ ]:
var_exp

In [ ]:
cum_var_exp  # in terms of percentage, inflated by 100 times

Build the projection matrix from the top two eigenvectors

In [ ]:
proj_mat = ((eig_vecs).T[:num_vec_to_keep]).T

In [ ]:
proj_mat.shape

In [ ]:
proj_mat

Project the data into the new data space

In [ ]:
pca_data = X_centered.dot(proj_mat)

In [ ]:
pca_data.shape

In [ ]:
pca_data

[OPTIONAL] Repeat the above eigen analysis using the Iris dataset <br>
Compare the results with those obtained with the `svd()` function in Section 1 of this practical

## 4. Manifold learning

*  In this section, we will experiment with Scikit-Learn's `KernelPCA` class to reduce the dimensionality of the famous Swiss roll dataset from 3 to 2 using the RBF and linear kernels. <br>
(Some parts of the code in this section are adopted from Reference [1])

Load the swiss roll dataset and examine it <br>
We get the dataset in `X`, and target values in `t`

In [ ]:
from sklearn.datasets import make_swiss_roll
X, t = make_swiss_roll(n_samples=1000, noise=0.2, random_state=42)

In [ ]:
X.shape

In [ ]:
X

In [ ]:
t.shape  # Target variable

In [ ]:
t

Perform Kernel PCA to reduce the dimenionality of the dataset <br>
Try the RBF kernel (assuming that we have used grid search and found that 0.0433  is the best gamma value to use)

In [ ]:
from sklearn.decomposition import KernelPCA

rbf_pca = KernelPCA(n_components=2, kernel="rbf", gamma=0.0433, fit_inverse_transform=True)
X_reduced = rbf_pca.fit_transform(X)
X_preimage = rbf_pca.inverse_transform(X_reduced)

In [ ]:
X_reduced.shape

In [ ]:
X_reduced

How does the swiss roll look like now in the 2-dimensional space?

In [ ]:
def plot_2d(data, target):
    plt.figure(figsize=(11, 4))
    plt.subplot(132)
    plt.scatter(data[:, 0], data[:, 1], c=target, cmap=plt.cm.hot, marker="x")
    plt.xlabel("$z_1$", fontsize=18)
    plt.ylabel("$z_2$", fontsize=18, rotation=0)
    plt.grid(True)

In [ ]:
plot_2d(X_reduced, t)

What is the reconstruction error?

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(X, X_preimage)

Now try using the linear kernel

In [ ]:
lin_pca = KernelPCA(n_components=2, kernel="linear", fit_inverse_transform=True)
X_reduced = lin_pca.fit_transform(X)
X_preimage = lin_pca.inverse_transform(X_reduced)

In [ ]:
plot_2d(X_reduced, t)

In [ ]:
mean_squared_error(X, X_preimage)

But `KernelPCA` with the linear kernel is simply equivalent to the `PCA` class

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X_reduced = pca.fit_transform(X)

In [ ]:
plot_2d(X_reduced, t)

#### References
[1] A. Geron (2017), Hands-on machine learning with Scikit-Learn and TensorFlow, Chapter 8 (O’Reilly). <br>
[2] G. Seif (2018), Principal Component Analysis: Your tutorial and code; https://towardsdatascience.com/principal-component-analysis-your-tutorial-and-code-9719d3d3f376.